# 07-3-3 Hugging Face 모델 활용

Colab에서 바로 실행 가능한 예제 모음입니다. 설치 → 기본 사용 → 다양한 파이프라인 → 챗봇 → (플러스) 파인튜닝 템플릿까지 포함합니다.

> ⚠️ 중요: 개인 **Access Token** 등 민감 정보는 절대 공개 저장소에 올리지 마세요.

## 07-3-3-1 Transformers 라이브러리

### (1) Transformers 라이브러리 설치 및 로그인

In [1]:
!pip install -q transformers

In [2]:
HF_TOKEN = "hf_xxx_your_token"

from huggingface_hub import login
login(token=HF_TOKEN)

### (2) 모델 로드 및 텍스트 생성 예제

In [5]:
from transformers import pipeline

# 텍스트 생성 모델 로드
generator = pipeline("text-generation", model="gpt2")

# 텍스트 생성
generated_text = generator("Hugging Face is", max_length=50, num_return_sequences=1)
print("\n\n=== 생성된 Text 결과\n")
print(generated_text)

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




=== 생성된 Text 결과

[{'generated_text': 'Hugging Face is the first of its kind, making it the fastest-growing video game franchise, according to the studio. Last year\'s blockbuster Splatoon was built over the course of two and a half years, and features the first full-fledged fighting game in the franchise. The studio has been working with Nintendo on the game for a while, and will give the game a test run in order to get an early look.\n\nExit Theatre Mode\n\n"It\'s a brand new thing for our team, and we\'re going to do the whole experience, that\'s for sure," said Mike Bresnik, game director at Splatoon, in a statement. "Right now we\'re just doing our best, trying to get this game out to the community as well as be as faithful as possible, and we\'re excited to share some of the core gameplay details with you guys."\n\nWhile the game is still in development, the studio will open up a few more locations for new players to explore.\n\n"We\'re definitely going to be open to a lot of di

### (3) 허깅페이스에서 제공하는 주요 모델
- **BERT** : 문장 임베딩 및 문장 이해에 최적화된 모델  
- **GPT-2** : 텍스트 생성 및 대화형 모델  
- **T5** : 텍스트 요약, 번역, 질의응답에 사용  
- **BART** : 텍스트 복원 및 생성 최적화 모델  

---
## 07-3-3-2 Hugging Face 모델 활용 예시

### 사용 예시 1: 감성 분석

In [6]:
from transformers import pipeline

# 감정 분석 (모델/리비전 명시 권장)
sent = pipeline(
    task="sentiment-analysis",
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english",
    revision="714eb0f"  # 예시 리비전
)
print(sent("Hugging Face is amazing!"))
print(sent("This is terrible and disappointing."))
print("-" * 30)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9998840093612671}]
[{'label': 'NEGATIVE', 'score': 0.999792754650116}]
------------------------------


### 사용 예시 2: 한국어 문장 번역 (ko→en)

In [7]:
from transformers import pipeline

# 한국어 → 영어 번역 (Helsinki-NLP/opus-mt-ko-en)
# 태스크는 'translation_ko_to_en' 지정 권장
translator = pipeline(
    task="translation_ko_to_en",
    model="Helsinki-NLP/opus-mt-ko-en",
)
print(translator("허깅페이스는 누구나 AI를 쉽게 쓰게 해줍니다."))

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/842k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


[{'translation_text': 'Huggingspace makes AI easy for everyone.'}]


### 사용 예시 3-1: 한국어 문장 요약 (AutoTokenizer / AutoModelForSeq2SeqLM)

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("psyche/KoT5-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("psyche/KoT5-summarization")

input_text = "오늘은 허깅페이스를 배우는 날입니다. 이 회사는 오픈소스 기반 AI 생태계로 유명합니다."
print("입력문장: \n" + input_text)
inputs = tokenizer("summarize: " + input_text, return_tensors="pt", max_length=512, truncation=True)

summary_ids = model.generate(inputs["input_ids"], max_length=100, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("\n요약문장:")
print(summary)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

입력문장: 
오늘은 허깅페이스를 배우는 날입니다. 이 회사는 오픈소스 기반 AI 생태계로 유명합니다.

요약문장:
이 회사는 오픈소스 기반 AI 생태계로 유명합니다.


### 사용 예시 3-2: 한국어 문장 요약 (pipeline)
모델 이름과 리비전을 명시하는 것이 좋습니다.

In [9]:
from transformers import pipeline

summarizer_pipeline = pipeline(
    "summarization",
    model="psyche/KoT5-summarization"
    # revision="..."  # 특정 리비전 고정 필요 시 주석 해제
)

input_text_for_pipeline = (
    "오늘은 허깅페이스를 배우는 날입니다. 파이프라인을 활용하면 요약, 번역, 질의응답 같은 다양한 작업을 쉽게 수행할 수 있습니다. "
    "이 회사는 오픈소스 기반 AI 생태계로 유명합니다.\n"
)
print("\n입력문장: \n" + input_text_for_pipeline)

summary_result_pipeline = summarizer_pipeline(
    input_text_for_pipeline,
    max_length=100,
    min_length=10,
    do_sample=False,
)
print("\n요약문장:")
print(summary_result_pipeline[0]['summary_text'])

Device set to use cpu
Your max_length is set to 100, but your input_length is only 46. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)



입력문장: 
오늘은 허깅페이스를 배우는 날입니다. 파이프라인을 활용하면 요약, 번역, 질의응답 같은 다양한 작업을 쉽게 수행할 수 있습니다. 이 회사는 오픈소스 기반 AI 생태계로 유명합니다.



Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



요약문장:
파이프라인을 활용하면 요약, 번역, 질의응답 같은 작업을 쉽게 수행할 수 있습니다. 이 회사는 오픈소스 기반 AI 생태계로 유명합니다.


### 사용 예시 4: 한국어 챗봇 (pipeline)

아래 셀은 필요한 의존성을 설치하고, 간단한 대화 이력 기반 챗봇을 구성합니다.

> 참고: Colab에서 길게 실행되는 `while True:` 루프는 수동 중지가 필요합니다. 필요 시 주석 처리 후 적절히 수정하세요.

In [10]:
!pip -q install "transformers>=4.44.0" sentencepiece gradio

In [15]:
import torch
from transformers import pipeline, AutoTokenizer

MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"
device = 0 if torch.cuda.is_available() else -1

tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
chatbot = pipeline(
    "text-generation",
    model=MODEL_NAME,
    tokenizer=tok,
    device=device
)

SYSTEM_PROMPT = (
    "You are a helpful, concise assistant for a university AI Agent course. "
    "Answer in the user's language (Korean or English)."
)

def format_history(history, user_text, system_prompt=SYSTEM_PROMPT):
    """
    history: list[tuple[str,str]] = [(user, assistant), ...]
    user_text: 현재 입력
    """
    lines = [f"System: {system_prompt}"]
    for u, a in history:
        lines.append(f"User: {u}")
        lines.append(f"Assistant: {a}")
    lines.append(f"User: {user_text}")
    lines.append("Assistant:")
    return "\n".join(lines)

def generate_reply(history, user_text):
    prompt = format_history(history, user_text)
    out = chatbot(
        prompt,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.05,
        pad_token_id=tok.eos_token_id
    )[0]["generated_text"]
    reply = out.split("Assistant:")[-1].strip()
    return reply[:1200]

# 🔁 (선택) 터미널 대화 루프 — 실행 시 수동 종료 필요
if True:
    history = []
    print("간단 챗봇 시작! 종료: exit/quit")
    while True:
        user = input("You: ").strip()
        if user.lower() in {"exit","quit"}:
            print("Bye!")
            break
        bot = generate_reply(history, user)
        history.append((user, bot))
        print("Bot:", bot)

Device set to use cpu


간단 챗봇 시작! 종료: exit/quit
You: 당신은 현명해요


KeyboardInterrupt: 

---
## 07-3-3-3 Hugging Face 모델 활용 플러스

### (1) Pretrained 모델 직접 로드 (Sequence Classification)

In [16]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

text = "Hugging Face makes NLP easy!"
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(probs)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tensor([[0.0013, 0.9987]], grad_fn=<SoftmaxBackward0>)


### (2) 모델 파인튜닝 (Fine-Tuning) 템플릿
아래 코드는 템플릿입니다. `train_dataset`/`eval_dataset`은 `datasets` 라이브러리 등으로 준비해야 합니다.

In [17]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
)

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
# )

# trainer.train()

### (3) Hugging Face Hub 업로드 & pipeline API 활용

In [18]:
# (참고) 모델을 Hub에 업로드하려면 먼저 로그인 필요
# !huggingface-cli login
# model.push_to_hub("my-huggingface-model")

from transformers import pipeline
classifier = pipeline("sentiment-analysis")
result = classifier("I love using Hugging Face!")
print(result)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9997085928916931}]


### (4) Datasets 로드 예시

In [19]:
!pip -q install datasets
from datasets import load_dataset
dataset = load_dataset("squad")
dataset

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})